In [1]:
from glob import glob

from tqdm import tqdm
import numpy as np

import g2s

RDKit WARNING: [15:09:54] Enabling RDKit 2019.09.3 jupyter extensions


In [3]:
ci_mols = sorted(glob('/Users/c0uch1/work/g2g_data/rxn_dset/min_xyz/*'))

In [4]:
# max_atoms = 0
# for i in tqdm(range(len(ci_mols))):
#     _, nc, _ = g2s.graph_extractor.xyz2mol_graph(ci_mols[i], get_chirality=False)
#     n_heavy_atoms = len(np.where(np.array(nc) != 1)[0])
#     max_atoms = n_heavy_atoms if n_heavy_atoms > max_atoms else max_atoms

In [5]:
representations = []
nuclear_charges = []
distances = []
local_hydrogen = []
heavy_hydrogen_mapping = []
hydrogen_heavy_dist = []
for i in tqdm(range(len(ci_mols))):
    bond_order_matrix, nc, coords = g2s.graph_extractor.xyz2mol_graph(ci_mols[i], get_chirality=False)
    dist = g2s.utils.calculate_distances(coords)
    gc = g2s.GraphCompound(bond_order_matrix, nc, dist)
    gc.filter_atoms('heavy')
    gc.generate_bond_length(size=15, sorting='norm-row')
    gc.generate_local_hydrogen_matrix()
    representations.append(gc.representation)
    nuclear_charges.append(gc.nuclear_charges)
    distances.append(gc.distances)
    local_hydrogen.append(gc.hydrogen_representations)
    heavy_hydrogen_mapping.append(gc.heavy_hydrogen_mapping)
    hydrogen_heavy_dist.append(gc.hydrogen_heavy_distances)

representations = np.array(representations)
nuclear_charges = np.array(nuclear_charges)
distances = np.array(distances)
local_hydrogen = np.array(local_hydrogen)
heavy_hydrogen_mapping = np.array(heavy_hydrogen_mapping)
hydrogen_heavy_dist = np.array(hydrogen_heavy_dist)

  0%|          | 0/1871 [00:00<?, ?it/s]/Users/c0uch1/github/Graph2Structure/g2s/representations/hydrogens.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(local_h_repr), np.array(heavy_hydrogen_mapping), np.array(hydrogen_heavy_distances)
100%|██████████| 1871/1871 [00:20<00:00, 92.01it/s] 
/Users/c0uch1/.miniconda3/envs/wk/lib/python3.7/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Users/c0uch1/.miniconda3/envs/wk/lib/python3.7/site-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning

In [6]:
train_kernel = g2s.krr.laplacian_kernel(representations, representations, 32)
train_kernel[np.diag_indices_from(train_kernel)] += 1e-5
alphas = g2s.krr.train_multikernel(train_kernel, distances)

pred_distances = g2s.krr.predict_distances(train_kernel, alphas)

100%|██████████| 105/105 [00:00<00:00, 484.27it/s]


In [7]:
hydrogen_representation = np.vstack(local_hydrogen)
hydrogen_distances = np.vstack(hydrogen_heavy_dist)

train_kernel = g2s.krr.laplacian_kernel(hydrogen_representation, hydrogen_representation, 32)
train_kernel[np.diag_indices_from(train_kernel)] += 1e-7
alphas = g2s.krr.train_multikernel(train_kernel, hydrogen_distances)

pred_hydrogen_distances = g2s.krr.predict_distances(train_kernel, alphas)

100%|██████████| 5/5 [00:00<00:00, 64.94it/s]


In [8]:
fpred_distances = g2s.utils.filter_nonzero_distances(pred_distances, nuclear_charges)

In [48]:
dgsol = g2s.dgeom.DGSOL(fpred_distances, nuclear_charges, vectorized_input=False)

In [49]:
dgsol.solve_distance_geometry('/Users/c0uch1/g2s_example/test')

100%|██████████| 1871/1871 [01:28<00:00, 21.16it/s]


In [23]:
import quadpy
import numpy as np


scheme = quadpy.sphere.lebedev_131()


def lebedev_sphere_opt(bond, points_distances, origin):
    """ Assumes that the bonded atom is in the origin, the other two are at pos1, pos2 with proton distances d1 and d2, respectively."""
    candidate_positions = scheme.points * bond
    deltas = np.sum(
        [(np.linalg.norm(candidate_positions - (pos - origin), axis=1) - dist) ** 2 for pos, dist in points_distances],
        axis=0)
    return candidate_positions[np.argmin(deltas)] + origin


def get_hydrogen_positions(positions, distances, n_hydrogens):
    h_h_distance = distances[-1]
    pos_1 = positions[0]
    dist_1 = distances[0]
    pos_hydr_1 = lebedev_sphere_opt(bond=dist_1, origin=pos_1, points_distances=zip(positions[1:], distances[1:-1]))
    if n_hydrogens > 1:
        p_dist_pairs = list(zip(positions[1:-1], distances[1:-2])) if len(positions) > 2 else list(zip(positions[1], distances[1]))
        p_dist_pairs.append((pos_hydr_1, h_h_distance))
        pos_hydr_2 = lebedev_sphere_opt(bond=dist_1, origin=pos_1, points_distances=p_dist_pairs)
    else:
        pos_hydr_2 = None
    if n_hydrogens == 3:
        p_dist_pairs = [(positions[1], distances[1]), (pos_hydr_1, h_h_distance), (pos_hydr_2, h_h_distance)]
        pos_hydr_3 = lebedev_sphere_opt(bond=dist_1, origin=pos_1, points_distances=p_dist_pairs)
    else:
        pos_hydr_3 = None
    return pos_hydr_1, pos_hydr_2, pos_hydr_3


def hydrogen_reconstruction(heavy_atom_coords, predicted_h_distances, heavy_atom_hydrogen_mapping):
    hydrogen_coords = []
    for i in range(len(heavy_atom_hydrogen_mapping)):
        n_hydrogens = len(heavy_atom_hydrogen_mapping[i][1])
        nbh_idxs = heavy_atom_hydrogen_mapping[i][2]
        center_atom_idx = heavy_atom_hydrogen_mapping[i][0]
        first_hydrogen_idx = heavy_atom_hydrogen_mapping[i][1][0]
        pred_positions = heavy_atom_coords[np.array([center_atom_idx, *nbh_idxs])]
        pred_h_distances = predicted_h_distances[i]

        pos_hydr_1, pos_hydr_2, pos_hydr_3 = get_hydrogen_positions(pred_positions, pred_h_distances, n_hydrogens=n_hydrogens)
        hydrogen_coords.append(pos_hydr_1)
        if pos_hydr_2 is not None:
            hydrogen_coords.append(pos_hydr_2)
        if pos_hydr_3 is not None:
            hydrogen_coords.append(pos_hydr_3)
    return np.array(hydrogen_coords)



In [148]:
for i, e in enumerate(sorted(glob('/Users/c0uch1/g2s_example/test/*'))):
    if heavy_hydrogen_mapping[i].size > 0.:
        h_coords = hydrogen_reconstruction(dgsol.coords[i], hydrogen_heavy_dist[i], heavy_hydrogen_mapping[i])
        heavy_c, all_nc = combine_heavy_hydrogen_coords(dgsol.coords[i],h_coords, nuclear_charges[i])
        write_xyz(f'{e}/dgsol_h.xyz', heavy_c, all_nc)
    else:
        write_xyz(f'{e}/dgsol_h.xyz', dgsol.coords[i], nuclear_charges[i])


6

In [42]:
from g2s.constants import periodic_table

def write_xyz(outname, coords, elements, element_input='nuclear_charges'):
    if element_input == 'nuclear_charges':
        elements = [periodic_table[int(nc)] for nc in elements]
    with open(f'{outname}', 'w') as outfile:
        outfile.write(f'{len(elements)}\n')
        outfile.write('\n')
        for xyz, nc in zip(coords, elements):
            outfile.write(f'{nc}\t{xyz[0]}\t{xyz[1]}\t{xyz[2]}\n')

def combine_heavy_hydrogen_coords(heavy_coords, hydrogen_coords, heavy_nuclear_charges):
    return np.vstack((heavy_coords, hydrogen_coords)), np.array([*heavy_nuclear_charges, *[1]*len(hydrogen_coords)])


In [81]:
from rdkit import Chem
from rdkit.Geometry import Point3D


In [160]:
def graph_to_rdkit(elements, adjacency_matrix):
    # Blatantly adapted from https://stackoverflow.com/questions/51195392/smiles-from-graph

    # create empty editable mol object
    mol = Chem.RWMol()

    # add atoms to mol and keep track of index
    node_to_idx = {}
    for i in range(len(elements)):
        a = Chem.Atom(elements[i])
        mol_idx = mol.AddAtom(a)
        node_to_idx[i] = mol_idx

    # add bonds between adjacent atoms
    for ix, row in enumerate(adjacency_matrix):
        for iy, bond in enumerate(row):

            # only traverse half the matrix
            if iy <= ix:
                continue

            # add relevant bond type (there are many more of these)
            if bond == 0:
                continue
            elif bond == 1:
                bond_type = Chem.rdchem.BondType.SINGLE
                mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
            elif bond == 2:
                bond_type = Chem.rdchem.BondType.DOUBLE
                mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
            elif bond == 3:
                bond_type = Chem.rdchem.BondType.Triple
                mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)

    # Convert RWMol to Mol object
    mol = mol.GetMol()
    Chem.SanitizeMol(mol)
    return mol


def embed_hydrogens(adjacency_matrix, nuclear_charges, heavy_atom_coords):
    elements = [periodic_table[nc] for nc in nuclear_charges]
    mol = graph_to_rdkit(elements, adjacency_matrix)

    # Generate some 2D coords, otherwise GetConformer is empty
    Chem.rdDepictor.Compute2DCoords(mol)
    conf = mol.GetConformer()

    # Set Coordinates
    for i in range(mol.GetNumAtoms()):
        x, y, z = heavy_atom_coords[i]
        conf.SetAtomPosition(i, Point3D(x, y, z))

    # Coord map fixes indices/coords during embedding
    coord_map = {i: mol.GetConformer().GetAtomPosition(i) for i in range(len(heavy_atom_coords))}
    mol_h = Chem.AddHs(mol)

    Chem.AllChem.EmbedMolecule(mol_h, coordMap=coord_map, useRandomCoords=True)

    embedded_coords = mol_h.GetConformer().GetPositions()
    embedded_nuclear_charges = [periodic_table.index(atom.GetSymbol()) for atom in mol_h.GetAtoms()]

    return embedded_coords, embedded_nuclear_charges


In [161]:
coords, nc = embed_hydrogens(gc.adjacency_matrix, gc.nuclear_charges, dgsol.coords[-1])

In [157]:
# from g2s.constants import periodic_table
# elements = [periodic_table[nc] for nc in gc.nuclear_charges]

In [163]:

write_xyz('/Users/c0uch1/g2s_example/test/1870/rdkit.xyz', coords , nc, element_input='nulear_charges')

In [162]:
coords

array([[ 1.33472675, -0.47439447,  1.43402831],
       [-1.30999509, -0.87824307,  2.04038499],
       [-0.71712658,  1.284657  ,  0.96379199],
       [-0.12279427, -0.11120553, -1.37436818],
       [-0.68718356, -2.16955979, -0.34991117],
       [-0.54396539, -0.16027251,  1.03044466],
       [-0.91056633, -0.74158746, -0.33940968],
       [-1.69016001, -0.17105456,  2.70462043],
       [-2.1022995 , -1.38372116,  1.60717842],
       [-0.43266785,  1.54768634,  0.01437858],
       [-0.10479872,  1.72244634,  1.70400413],
       [ 0.83031574,  0.11652555, -1.03753139],
       [-0.60311797,  0.68691482, -1.82675536],
       [ 0.16218357, -2.41359295,  0.2040279 ],
       [-1.49303972, -2.71883865,  0.00307187],
       [-1.95977745, -0.48013388, -0.50777005]])

In [144]:
nc

[35, 7, 7, 7, 7, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1]